<a href="https://colab.research.google.com/github/ykato27/Object-Detection/blob/main/NanoDet_Colaboratory_Training_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Lightningインストール(PyTorch Lightning install)

In [1]:
!pip install pytorch_lightning

     |████████████████████████████████| 523 kB 4.1 MB/s 
     |████████████████████████████████| 329 kB 36.4 MB/s 
     |████████████████████████████████| 596 kB 35.4 MB/s 
     |████████████████████████████████| 829 kB 41.7 MB/s 
     |████████████████████████████████| 132 kB 44.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.0 MB/s 
     |████████████████████████████████| 160 kB 47.2 MB/s 
     |████████████████████████████████| 271 kB 49.1 MB/s 
     |████████████████████████████████| 192 kB 52.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=a4a180ded0efa1b6f9042fda005a29e074369b804de485296b3c3fb58379d08f
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exis

# 乱数シード固定(Random seed fixed)

In [2]:
import os
import random

import torch
import numpy as np
import glob
import shutil

def initialize_random_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:
initialize_random_seed(42)

# データセットダウンロード(Download Dataset)

In [4]:
use_sample_image = True

if use_sample_image:
    !git clone https://github.com/Kazuhito00/NanoDet-Colaboratory-Training-Sample.git

Cloning into 'NanoDet-Colaboratory-Training-Sample'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 195 (delta 68), reused 169 (delta 54), pack-reused 0
Receiving objects: 100% (195/195), 29.95 MiB | 14.17 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [5]:
# 独自のデータを使用する場合は、パスを指定してください
# Please fill in the path if you want to use your own data
if use_sample_image:
    dataset_directory = 'NanoDet-Colaboratory-Training-Sample/02.annotation_data'
else:
    dataset_directory = ''

# 学習/検証データパス
train_directory = './train'
validation_directory = './validation'

# 学習データ格納ディレクトリ作成(Create training data storage directory)
for dir_path in os.listdir(dataset_directory):
    os.makedirs(train_directory + '/' + dir_path, exist_ok=True)
# 検証データ格納ディレクトリ作成(Create verification data storage directory)
for dir_path in os.listdir(dataset_directory):
    os.makedirs(validation_directory + '/' + dir_path, exist_ok=True)

In [6]:
# 学習データの割合(Percentage of training data)
train_ratio = 0.8

# コピー元ファイルリスト取得(Get copy source file list)
annotation_list = sorted(glob.glob(dataset_directory + '/Annotations/*'))
image_list = sorted(glob.glob(dataset_directory + '/JPEGImages/*'))

file_num = len(annotation_list)

# インデックスシャッフル(shuffle)
index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(index_list):
    if count < int(file_num * train_ratio):
        # 学習用データ(Training Data)
        shutil.copy2(annotation_list[index], train_directory + '/Annotations')
        shutil.copy2(image_list[index], train_directory + '/JPEGImages')
    else:
        # 検証用データ(Validation Data)
        shutil.copy2(annotation_list[index], validation_directory + '/Annotations')
        shutil.copy2(image_list[index], validation_directory + '/JPEGImages')

# Pascal VOC形式 を MS COCO形式へ変換

In [7]:
!git clone https://github.com/Kazuhito00/voc2coco.git

Cloning into 'voc2coco'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 53 (delta 0), reused 1 (delta 0), pack-reused 50
Unpacking objects: 100% (53/53), done.


In [8]:
!python voc2coco/voc2coco.py train/Annotations train/annotations.json
!python voc2coco/voc2coco.py validation/Annotations validation/annotations.json

Number of xml files: 40
Success: train/annotations.json
Number of xml files: 9
Success: validation/annotations.json


# モデル訓練(Training Model)

In [9]:
!git clone https://github.com/RangiLyu/nanodet.git
!cp NanoDet-Colaboratory-Training-Sample/03.config/nanodet-m.yml nanodet/nanodet-m.yml

Cloning into 'nanodet'...
remote: Enumerating objects: 2271, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 2271 (delta 34), reused 71 (delta 24), pack-reused 2162
Receiving objects: 100% (2271/2271), 5.27 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (1281/1281), done.


In [10]:
%cd nanodet
!cp tools/train.py ./

/content/nanodet


「nanodet-m.yml」を格納してください(Store "nanodet-m.yml")<br><br>
![image](https://user-images.githubusercontent.com/37477845/133949475-e6aefaab-fbcd-41fd-a059-b99999bbe5a3.png)

In [11]:
# 訓練(Training)
!python train.py nanodet-m.yml

[NanoDet][11-26 00:09:51]INFO:Setting up data...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
[NanoDet][11-26 00:09:51]INFO:Creating model...
model size is  1.0x
init weights...
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth
100% 8.79M/8.79M [00:00<00:00, 64.0MB/s]
=> loading pretrained model https://download.pytorch.

# ONNX変換(Convert to ONNX)

In [12]:
!cp tools/export_onnx.py ./

In [13]:
!python export_onnx.py \
    --cfg_path=nanodet-m.yml \
    --model_path=workspace/nanodet_m/model_best/model_best.ckpt

model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet Head.
/content/nanodet/nanodet/model/backbone/shufflenetv2.py:18: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  channels_per_group = num_channels // groups
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mod

In [14]:
!pip install onnx-simplifier

     |████████████████████████████████| 12.7 MB 4.2 MB/s 
     |████████████████████████████████| 466 kB 36.4 MB/s 
     |████████████████████████████████| 4.8 MB 17.5 MB/s 
  Created wheel for onnx-simplifier: filename=onnx_simplifier-0.3.6-py3-none-any.whl size=12873 sha256=07530e3c350408aab84b56b2012a696e293cde8ddd6993614d3fc596cd04b307
  Stored in directory: /root/.cache/pip/wheels/0c/47/80/8eb21098e22c19d60b1c14021ee67442b4ad2d7991fdad46ba
Successfully built onnx-simplifier


In [15]:
!python -m onnxsim nanodet.onnx nanodet.onnx

Simplifying...
Checking 0/3...
Checking 1/3...
Checking 2/3...
Ok!


# ONNXファイル情報確認(Check ONNX file information)

In [16]:
!pip install onnxruntime

In [17]:
import onnxruntime

onnx_session = onnxruntime.InferenceSession('nanodet.onnx')

In [18]:
input_detail = onnx_session.get_inputs()
output_detail = onnx_session.get_outputs()

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name

print(len(input_detail), len(output_detail))
print('input_detail: ', input_detail[0])
print('output_detail: ', output_detail[0])
print('output_detail: ', output_detail[1])
print('output_detail: ', output_detail[2])
print('output_detail: ', output_detail[3])
print('output_detail: ', output_detail[4])
print('output_detail: ', output_detail[5])

1 6
input_detail:  NodeArg(name='input.1', type='tensor(float)', shape=[1, 3, 320, 320])
output_detail:  NodeArg(name='cls_pred_stride_8', type='tensor(float)', shape=[1, 1600, 1])
output_detail:  NodeArg(name='cls_pred_stride_16', type='tensor(float)', shape=[1, 400, 1])
output_detail:  NodeArg(name='cls_pred_stride_32', type='tensor(float)', shape=[1, 100, 1])
output_detail:  NodeArg(name='dis_pred_stride_8', type='tensor(float)', shape=[1, 1600, 32])
output_detail:  NodeArg(name='dis_pred_stride_16', type='tensor(float)', shape=[1, 400, 32])
output_detail:  NodeArg(name='dis_pred_stride_32', type='tensor(float)', shape=[1, 100, 32])


# 学習済ファイルダウンロード(Download Trained Model)

In [19]:
!cp 'nanodet.onnx' 'workspace/nanodet_m'

In [20]:
# ダウンロードする
from google.colab import files

!zip -r 'workspace.zip' 'workspace'
files.download('workspace.zip')

  adding: workspace/ (stored 0%)
  adding: workspace/nanodet_m/ (stored 0%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/ (stored 0%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/Train_loss_lr_Train/ (stored 0%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/Train_loss_lr_Train/events.out.tfevents.1637885405.2716267a1b6a.177.1 (deflated 59%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/Val_metrics_mAP_Val/ (stored 0%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/Val_metrics_mAP_Val/events.out.tfevents.1637885421.2716267a1b6a.177.5 (deflated 60%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/train_cfg.yml (deflated 62%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/Val_metrics_AP_50_Val/ (stored 0%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/Val_metrics_AP_50_Val/events.out.tfevents.1637885421.2716267a1b6a.177.6 (deflated 62%)
  adding: workspace/nanodet_m/logs-2021-11-26-00-09-51/events.out.tfevents.163788540

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>